In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns
from pandas import DataFrame, Series
import scipy.stats as scst
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, mean_squared_error, r2_score
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import OneClassSVM, SVC
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error as mae
import time
import xgboost as xgb

from encode_many import encode_many
from scale_it import scale_it
from unscale import unscale


In [ ]:
# load the data
# path = 'string'
path = 'final_patient_data.xlsx'
PPdata = pd.read_excel(r'/Users/fardadddii/Documents/Hospital data/code/final_patient_data.xlsx')

In [ ]:
##cleaning
# checking number of NANs
PPdata.isna().sum()
# filling NANs
PPdata['SOURCE_OF_ADMISSION'].fillna(0, inplace = True)
# drop with thresshol
PPdata.dropna(thresh=2)
#duplicate data
PPdata.drop_duplicates()

In [ ]:
#meaningless mis-recorded dat
ADI_neg_droped_index = PPdata[PPdata['ADMISSION_TO_DEATH_INTERVAL']<0].index
PPdata.drop(index = PPdata[PPdata['ADMISSION_TO_DEATH_INTERVAL']<0].index, inplace =True)
PPdata = PPdata.reset_index(drop=True)

In [ ]:
# initial visualization of Race distribusion (other features could be passed)
feat = 'RACE'
g = sns.countplot(PPdata[feat], color = 'seagreen')
# g.set_xticklabels(['White','Black', 'Other', 'Asian', 'Hispanic'])
#g.set_xticklabels(['White','Black', 'Hispanic'])
for p in g.patches:
    g.annotate('{:.1f}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+100))
plt.title('Distribution of patient: '+feat)
plt.show()


In [ ]:
#Hot Encoding

PPdata = PPdata.join(encode_many(PPdata,'RACE'))   
PPdata = PPdata.join(encode_many(PPdata,'AGE'))   
PPdata = PPdata.join(encode_many(PPdata,'MEDICARE_STATUS_CODE'))   
PPdata = PPdata.join(encode_many(PPdata,'TYPE_OF_ADMISSION'))   
PPdata = PPdata.join(encode_many(PPdata,'SOURCE_OF_ADMISSION'))   

##Binary Encoding
le = preprocessing.LabelEncoder()
le.fit(PPdata['DISCHARGE_STATUS'])
PPdata['DISCHARGE_STATUS_BINARY'] = (le.transform(PPdata['DISCHARGE_STATUS']))

# Sex Encoder
le = preprocessing.LabelEncoder()
le.fit(PPdata['SEX'])
PPdata['SEX_BINARY'] = (le.transform(PPdata['SEX']))

In [ ]:
# Final SVM tuning with nested cross validation and feature importance
# This in-house code is faster than GridSearchCV. We have a grid search tuning at the end of this code

data = PPdata[['CCI','Race1', 'Race2', 'Race3', 'Race4',
       'Race5', 'SEX_BINARY', 'Age1', 'Age2', 'Age3', 'Age4', 'Age5', 'Age6',
       'Age7', 'Age8', 'medicale_status_code1',
       'medicale_status_code2', 'medicale_status_code3',
       'medicale_status_code4', 'medicale_status_code5', 'type_of_admission1',
       'type_of_admission2', 'type_of_admission3', 'type_of_admission4']].copy()

#data = PPdata[['CCI_POA','Race1', 'Race2', 'Race3', 'Race4',
#      'Race5']].copy()

label = PPdata[['ICU']].copy()

# scaler

#_, min_icu, max_icu = scale_it(label,'ICU')

_, min_cci, max_cci  = scale_it(data,'CCI_POA')

## function for unscalaing

x_data = data.to_numpy()

y_data = label.to_numpy()

cv_in = 5
cv_out = 10
rand = 1

kernel_rng = ['poly', 'rbf', 'sigmoid']
gamma_rng = ['scale', 'auto']
degree_rng = [2,3,4,5]
C_rng = [0.1, 1, 5, 10, 20]
#C_rng = [0.001,1]

indexer = 0
indexer_out = 0
elapsed = 0

cv_inner = KFold(n_splits=cv_in, shuffle=True, random_state=42)
cv_outer = KFold(n_splits=cv_out, shuffle=True, random_state=42)

outer_results = pd.DataFrame(columns = ['outer_fold', 'kernel', 'degree', 'C_reg','gamma', 'mae_test', 'mae_train'])

fi_mx = []
fi_feat = []
fi_comined = []
t = time.time()

for cv_ind_outer, (train_out, test_out) in enumerate(cv_outer.split(x_data,y_data)):
    
    optimization_folds = pd.DataFrame(columns = ['inner_Fold', 'kernel', 'degree', 'C_reg','gamma', 'mae_test'])
    
    x_train_out = x_data[train_out]
    y_train_out = y_data[train_out]
    
    x_test_out = x_data[test_out]
    y_test_out = y_data[test_out]
    
    for cv_ind_inner, (train_in, test_in) in enumerate(cv_inner.split(x_train_out,y_train_out)):
        
        optimization_par = pd.DataFrame(columns = ['inner_Fold', 'kernel', 'degree', 'C_reg','gamma', 'mae_test'])
        
        print(f'inner fold number  {cv_ind_inner}')
        
        t = time.time()
        
        x_train_in = x_train_out[train_in]
        y_train_in = y_train_out[train_in]
        
        x_test_in = x_train_out[test_in]
        y_test_in = y_train_out[test_in]
                           
        for kr_ind, krn in enumerate(kernel_rng):

            for deg in degree_rng:

                for Cr in C_rng:

                    for gm_ind,gam in enumerate(gamma_rng):

                        mlp = SVR(C=Cr, kernel=krn, degree=deg, gamma=gam)

                        mlp.fit(x_train_in, y_train_in)

                        y_pred_in = mlp.predict(x_test_in)
                        #score = mlp.score(x_test_in,y_test_in)

                        y_test_in_unscaled = unscale(y_test_in)
                        y_pred_in_unscaled = unscale(y_pred_in)
                        
                        
                        perf_unscaled = mae(y_test_in_unscaled,y_pred_in_unscaled)
                        perf = mae(y_test_in,y_pred_in)
                        
                        
                        print(perf)

                        optimization_par.loc[indexer,:] = [cv_ind_inner, kr_ind, deg, Cr, gm_ind, perf]

                        indexer+=1

        min_optimized = optimization_par[optimization_par['mae_test'] == np.min(optimization_par['mae_test'])]

        optimization_folds = optimization_folds.append(min_optimized, ignore_index=True)

    print(f'outer_fold number {cv_ind_outer}')
    
    print(f'inner folds optimized values \n \n {optimization_folds}')
    
    modes = optimization_folds['kernel'].mode()
    temp = []
    for m in modes:
        temp.append(optimization_folds[optimization_folds['kernel']==m]['mae_test'].min())
    kr_op = modes[np.where(np.array(temp)==np.array(temp).min())[0][0]]
   
    modes = optimization_folds['degree'].mode()
    temp = []
    for m in modes:
        temp.append(optimization_folds[optimization_folds['degree']==m]['mae_test'].min())
    deg_op = modes[np.where(np.array(temp)==np.array(temp).min())[0][0]]
   
    modes = optimization_folds['C_reg'].mode()
    temp = []
    for m in modes:
        temp.append(optimization_folds[optimization_folds['C_reg']==m]['mae_test'].min())
    Cr_op = modes[np.where(np.array(temp)==np.array(temp).min())[0][0]]
    
    modes = optimization_folds['gamma'].mode()
    temp = []
    for m in modes:
        temp.append(optimization_folds[optimization_folds['gamma']==m]['mae_test'].min())
    gamma_op = modes[np.where(np.array(temp)==np.array(temp).min())[0][0]]
    
    mlp1 = SVR(C=Cr_op, kernel=kernel_rng[kr_op], degree=deg_op, gamma=gamma_rng[gamma_op])
    
    mlp1.fit(x_train_out, y_train_out)
    
    ypred_test_out = mlp1.predict(x_test_out)
    ypred_test_out_unscaled = unscale(ypred_test_out)
    
    
    ypred_train_out = mlp1.predict(x_train_out)
    ypred_train_out_unscaled = unscale(ypred_train_out)
    
    y_train_out_unscaled = unscale(y_train_out)
    y_test_out_unscaled = unscale(y_test_out)
    
    mae_train_out_unscaled = mae (y_train_out_unscaled, ypred_train_out_unscaled)
    
    mae_test_out_unscaled = mae (y_test_out_unscaled, ypred_test_out_unscaled)
    
    mae_train_out = mae (y_train_out, ypred_train_out)
    
    mae_test_out = mae (y_test_out, ypred_test_out)
    
    print(mae_test_out)
    
    outer_results.loc[indexer_out,:] = [cv_ind_outer, kernel_rng[kr_op], deg_op, Cr_op, gamma_rng[gamma_op],mae_test_out, mae_train_out]
    indexer_out+=1

    elapsed = time.time() - t

    time_estimate = elapsed*(10 - cv_ind_outer)

    print(f'remaining time is {time_estimate/60} minutes')

    print (f'omtimized values are \n \n {outer_results}')

optimization_par  # the whole parameter variation dataframe


In [ ]:
##XGBOOSt # grid search tuning with nested cross validation####

data = PPdata[['RACE', 'CCI', 'SEX', 'AGE', 'TYPE_OF_ADMISSION', 'MEDICARE_STATUS_CODE']]

lable = PPdata['ICU']

# x_data = PPdata[['CCI_POA','RACE']].copy()

# y_data = PPdata[['ICU']].copy()

#_, min_icu, max_icu = scale_it(label,'ICU')

_, min_cci, max_cci  = scale_it(x_data,'CCI_POA')

## function for unscalaing

data = x_data
label = y_data

eval_metric_name = 'mae'

#del optimization_par

optimization_par = pd.DataFrame(columns = ['Fold', 'depth', 'gamma', 'eta', 'min_child_weight','subsample','objective', 'min_eval', 'mae_test', 'mae_train'])

optimization_folds = pd.DataFrame(columns = ['Fold', 'depth', 'gamma', 'eta', 'min_child_weight','subsample','objective', 'min_eval', 'mae_test', 'mae_train'])

objective_funcs = ['reg:squarederror', 'reg:squaredlogerror', 'reg:pseudohubererror']

learning_rate = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.5]

three_depth = range(2,16)

#gamme_value = np.linspace(0,10,2, endpoint = True)
gamme_value = [0, 5, 10]

subsample_values = [0.3, 0.5, 0.9]

min_child_weight = [1, 5, 10]
indexer = 0
indexer_out = 0
elapsed = 0

cv_outer = KFold(n_splits = 10, shuffle=True, random_state=42)
cv_in = KFold(n_splits = 5, shuffle=True, random_state=42)

#optimization_folds = []

outer_results = pd.DataFrame(columns = ['OuterFold', 'depth', 'gamma', 'eta', 'min_child_weight','subsample','objective', 'min_eval_outer', 'mae_test_outer', 'mae_train_outer'])

fi_mx = []
fi_feat = []
fi_comined = []

param = {}

for cv_ind_inner, (train_in, test_in) in enumerate(cv_in.split(data, lable)):

    print(f'inner fold number  {cv_ind_inner}')

    t = time.time()

    X_train_in = data.iloc[train_in]
    y_train_in = lable.iloc[train_in]

    X_test_in = data.iloc[test_in]
    y_test_in = lable.iloc[test_in]

    dtest_in = xgb.DMatrix(X_test_in, label=y_test_in)
    dtrain_in = xgb.DMatrix(X_train_in, label=y_train_in)
    optimization_par = pd.DataFrame(columns = ['Fold', 'depth', 'gamma', 'eta', 'min_child_weight','subsample','objective', 'min_eval', 'mae_test', 'mae_train'])

    for dp_id, max_depth_input in enumerate(three_depth):

        print(f'depth is {max_depth_input}')

        for gamma_val in gamme_value:

            for eta_val in learning_rate:

                for reg_ind, reg in enumerate(objective_funcs):

                    for subs in subsample_values:

                        for ch_w in min_child_weight:

                            param = {'max_depth': max_depth_input, 'objective': reg, 'eval_metric' : eval_metric_name}
                            param['nthread'] = 4
                            #param['eval_metric'] = 'auc'
                            param['booster'] = 'gbtree'
                            param['validate_parameters'] = True
                            param['gamma'] = gamma_val
                            param['eta'] = eta_val   

                            param['subsample'] = subs

                            param['min_child_weight'] = ch_w

                            evallist = [(dtest_in, 'eval'), (dtrain_in, 'train')]

                            num_round = 1000

                            np.array

                            evals_result = {}
                            bst = xgb.train(param, dtrain_in, num_round, verbose_eval = False, evals=evallist, early_stopping_rounds=10, evals_result=evals_result)

                            ypred_test_in = bst.predict(dtest_in)
                            y_test_in

                            ypred_train_in = bst.predict(dtrain_in)
                            y_train_in

                            mae_train = mean_absolute_error (y_train_in, ypred_train_in)

                            mae_test = mean_absolute_error (y_test_in, ypred_test_in)
                            
                            print(mae_test)

                            min_eval_inner = np.min(evals_result['eval'][eval_metric_name])

                            optimization_par.loc[indexer,:] = [cv_ind_inner, max_depth_input, gamma_val, eta_val, ch_w, subs, reg_ind, min_eval_inner, mae_test, mae_train]

                            indexer+=1

    min_optimized = optimization_par[optimization_par['mae_test'] == np.min(optimization_par['mae_test'])]

    optimization_folds = optimization_folds.append(min_optimized, ignore_index=True)
    fi_score = bst.get_score (importance_type = 'cover')
    
    fi_comined.append(fi_score)

    fi_mx.append(list(fi_score.values()))

    fi_feat.append(list(fi_score.keys()))
    
print(optimization_folds)

feature_importance = DataFrame(fi_mx, columns = fi_feat[0])

feature_importance_ms = DataFrame(np.array([list(feature_importance.mean()), list(feature_importance.std())]), index = ['mean', 'std'], columns = fi_feat[0]).T
feature_importance_ms.sort_values(by=['mean'], ascending = True, inplace = True)

font = {'family' : 'arial',
        'weight' : 'bold',
        'size'   : 15}
mpl.rc('font', **font)
plt.barh(list(feature_importance_ms.index),list(feature_importance_ms['mean']) ,  xerr = list(feature_importance_ms['std']), color = 'g')



In [ ]:
# permutation based feature importance

X = data

#_, min_cci, max_cci  = scale_it(X,'CCI')

r = r_multi['neg_mean_absolute_error']

CCI_imp = abs(r.importances[0])

sex_imp = abs(r.importances[6])

race_imp = sum(abs(r.importances[1:5]))

age_imp = sum(abs(r.importances[7:14]))

medicare_imp = sum(abs(r.importances[15:19]))

type_imp = sum(abs(r.importances[20:23]))

columns_imp = ['CCI', 'Sex', 'Race', 'Age', 'Medicare OREC', 'Type of admission']

var_mean = [CCI_imp.mean(), sex_imp.mean(), race_imp.mean(), age_imp.mean(), medicare_imp.mean(), type_imp.mean()]

feature_importances = DataFrame(var_mean, index = columns_imp)

feature_importances_sorted = feature_importances.sort_values(by = 0, ascending = True)

font = {'family' : 'arial',
        'weight' : 'bold',
        'size'   : 15}
mpl.rc('font', **font)

plt.barh( list(feature_importances_sorted.index), list(feature_importances_sorted[0]), color = 'k')

In [ ]:
##Working Regression XGBOOSt # optimization with nested cross validation####

data = PPdata[['RACE', 'CCI', 'SEX', 'AGE', 'TYPE_OF_ADMISSION', 'MEDICARE_STATUS_CODE']]

lable = PPdata['ICU']

eval_metric_name = 'mae'

optimization_par = pd.DataFrame(columns = ['Fold', 'depth', 'gamma', 'eta', 'min_child_weight','subsample','objective', 'min_eval', 'mae_test', 'mae_train'])

optimization_folds = pd.DataFrame(columns = ['Fold', 'depth', 'gamma', 'eta', 'min_child_weight','subsample','objective', 'min_eval', 'mae_test', 'mae_train'])

objective_funcs = ['reg:squarederror', 'reg:squaredlogerror', 'reg:pseudohubererror']
#objective_funcs = ['reg:squarederror']

learning_rate = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.5]

three_depth = range(6,7)
#three_depth = [6]

gamme_value = np.linspace(0,10,2, endpoint = True)
#gamme_value = [0, 5, 10]

subsample_values = [0.3, 0.5, 0.9]

min_child_weight = [1, 5, 10]


indexer = 0
indexer_out = 0
elapsed = 0

cv_outer = KFold(n_splits = 10, shuffle=True, random_state=42)
cv_in = KFold(n_splits = 5, shuffle=True, random_state=42)

outer_results = pd.DataFrame(columns = ['OuterFold', 'depth', 'gamma', 'eta', 'min_child_weight','subsample','objective', 'min_eval_outer', 'mae_test_outer', 'mae_train_outer'])

fi_mx = []
fi_feat = []
fi_comined = []

for cv_ind_outer, (train_out, test_out) in enumerate(cv_outer.split(data, lable)):
    
    X_train_out = data.iloc[train_out]
    y_train_out = lable.iloc[train_out]
    
    X_test_out = data.iloc[test_out]
    y_test_out = lable.iloc[test_out]
    
    dtest_out = xgb.DMatrix(X_test_out, label=y_test_out)
    dtrain_out = xgb.DMatrix(X_train_out, label=y_train_out)
    
    param = {}
    
    for cv_ind_inner, (train_in, test_in) in enumerate(cv_in.split(X_train_out, y_train_out)):
        
        print(f'inner fold number  {cv_ind_inner}')
        
        t = time.time()
        
        X_train_in = data.iloc[train_in]
        y_train_in = lable.iloc[train_in]
        
        X_test_in = data.iloc[test_in]
        y_test_in = lable.iloc[test_in]
        
        dtest_in = xgb.DMatrix(X_test_in, label=y_test_in)
        dtrain_in = xgb.DMatrix(X_train_in, label=y_train_in)
        
        for dp_id, max_depth_input in enumerate(three_depth):
            
            print(f'depth is {max_depth_input}')
            
            for gamma_val in gamme_value:

                for eta_val in learning_rate:

                    for reg_ind, reg in enumerate(objective_funcs):
                        
                        for subs in subsample_values:
                            
                            for ch_w in min_child_weight:

                                param = {'max_depth': max_depth_input, 'objective': reg, 'eval_metric' : eval_metric_name}
                                param['nthread'] = 4
                                param['booster'] = 'gbtree'
                                param['validate_parameters'] = True
                                param['gamma'] = gamma_val
                                param['eta'] = eta_val 

                                param['subsample'] = subs 

                                param['min_child_weight'] = ch_w 

                                evallist = [(dtest_in, 'eval'), (dtrain_in, 'train')]

                                num_round = 1000

                                np.array

                                evals_result = {}
                                bst = xgb.train(param, dtrain_in, num_round, verbose_eval = False, evals=evallist, early_stopping_rounds=10, evals_result=evals_result)

                                ypred_test_in = bst.predict(dtest_in)
                                y_test_in

                                ypred_train_in = bst.predict(dtrain_in)
                                y_train_in

                                mae_train = mean_absolute_error (y_train_in, ypred_train_in)

                                mae_test = mean_absolute_error (y_test_in, ypred_test_in)

                                min_eval_inner = np.min(evals_result['eval'][eval_metric_name])

                                optimization_par.loc[indexer,:] = [cv_ind_inner, max_depth_input, gamma_val, eta_val, ch_w, subs, reg_ind, min_eval_inner, mae_test, mae_train]

                                indexer+=1

        min_optimized = optimization_par[optimization_par['mae_test'] == np.min(optimization_par['mae_test'])]

        optimization_folds = optimization_folds.append(min_optimized, ignore_index=True)

    elapsed = time.time() - t

    time_estimate = elapsed*(10 - cv_ind_outer)

    print(f'remaining time is {time_estimate/60} minutes')

    print(f'outer_fold number {cv_ind_outer}')
    
    print(f'optimized values \n {min_optimized}')
    
    max_depth_input_op = int(round(optimization_folds['depth'].mean()))
    
    eta_val_op = float(optimization_folds.loc[-4:,['eta']].mean())
    
    gamma_val_op = int(optimization_folds.loc[-4:,['gamma']].mean())
        
    sub_op = float(optimization_folds.loc[-4:,['subsample']].mean())
        
    mc_w_op = int(round(optimization_folds.loc[-4:,['min_child_weight']].mean()))

    param = {'max_depth': max_depth_input_op, 'objective': reg, 'eval_metric' : eval_metric_name}

    param['gamma'] = gamma_val_op

    param['eta'] = eta_val_op
    
    param['subsample']  = sub_op
    
    param['min_child_weight'] = mc_w_op
    
    evallist_out = [(dtest_out, 'eval'), (dtrain_out, 'train')]
    
    eval_results_out = {}

    bst1 = xgb.train(param, dtrain_out, num_round, verbose_eval = False, evals=evallist_out, early_stopping_rounds=10, evals_result=eval_results_out)

    ypred_test_out = bst1.predict(dtest_out)
    
    ypred_train_out = bst1.predict(dtrain_out)
    
    mae_train_out = mean_absolute_error (y_train_out, ypred_train_out)
    
    mae_test_out = mean_absolute_error (y_test_out, ypred_test_out)
    
    min_eval_out = np.min(eval_results_out['eval'][eval_metric_name])

    outer_results.loc[indexer_out,:] = [cv_ind_outer, max_depth_input_op, gamma_val_op, eta_val_op, mc_w_op, sub_op, reg_ind, min_eval_out, mae_test_out, mae_train_out]
    indexer_out+=1
    
    print(f'outer results are \n {outer_results}')
    
    fi_score = bst1.get_score (importance_type = 'cover')
    
    fi_comined.append(fi_score)

    fi_mx.append(list(fi_score.values()))

    fi_feat.append(list(fi_score.keys()))
    
    print(f'score values {fi_mx}')
    
    print(f'score keys {fi_feat}')
     
optimization_folds # Optimization Variables
    
min_folds_params = optimization_folds [np.where(optimization_folds == np.array(optimization_folds)[:,5].min())[0][0],:] #optimized parameters of the kfold cross validation

fueatute_imo = DataFrame(fi_mx, columns = list(fi_score.keys())) #feature importance for the folds

optimization_par  # the whole parameter variation dataframe

print (f'omtimized values are {min_folds_params}')

font = {'family' : 'arial',
        'weight' : 'bold',
        'size'   : 13}
mpl.rc('font', **font)
imp = xgb.plot_importance(bst, height=0.8, xlim=None, ylim=None, fmap='', importance_type='cover', max_num_features=None, grid=False, show_values=False, color = 'k')
imp = imp.plot(color='k')
#plt.title('Feature Importance (wieght)',weight='bold')
plt.title(None)
#plt.xlabel('F Score',weight='bold')
plt.xlabel(None)
plt.ylabel(None)
plt.show()

In [ ]:

x_data = PPdata[['RACE', 'CCI_POA', 'SEX', 'AGE', 'TYPE_OF_ADMISSION', 'MEDICARE_STATUS_CODE']]

y_data = PPdata['ICU']
    
#compute optimal params

# fit final model with optimal params
mlp_opt = SVR(C=10, kernel='rbf', degree=1, gamma='auto').fit(x_data,y_data)

# begin feature importance eval
feat_names = np.array(["AGE","SEX","RACE","CCI","TOA","OREC"])
scoring = ['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error']
r_multi = permutation_importance(mlp_opt,x_data,y_data,n_repeats=10,random_state=42,scoring=scoring)

for metric in r_multi:
    print(f"{metric}")
    r = r_multi[metric]
    
    for i in r.importances_mean.argsort()[::-1]:
        
        if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
            print(f"    {feat_names[i]:<8}"
            f"{r.importances_mean[i]:.3f}"
            f" +/- {r.importances_std[i]:.3f}")

In [ ]:
X = PPdata[["AGE","SEX","RACE","CCI","TYPE_OF_ADMISSION","MEDICARE_STATUS_CODE"]]

r = r_multi['neg_mean_absolute_error']

sorted_importances_idx = r.importances_mean.argsort()
importances = pd.DataFrame(
    r.importances[sorted_importances_idx].T,columns=X.columns[sorted_importances_idx],)

test = importances.mean().sort_values(ascending=True)

ax = test.plot.barh()
#ax.set_title("Permutation Importances (test set)",fontdict=titledict)
ax.set_xlabel("Decrease in accuracy score",fontdict= {'fontsize':20,'fontweight':'bold'})
ax.set_yticklabels(test.index,size = 15,weight='bold')
ax.set_xticklabels(ax.get_xticks(),size = 15,weight='bold')
#ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))

In [ ]:
## XGBOOST grid search with repeated stratified kfold validation
data = PPdata[['RACE', 'CCI', 'SEX', 'AGE', 'TYPE_OF_ADMISSION', 'MEDICARE_STATUS_CODE']]

lable = PPdata['ICU']

model = XGBRegressor()
cv_inner= KFold(n_splits=5, shuffle=True, random_state=42)

space = dict()
space['objective'] = ['reg:squarederror']
space['max_depth'] = [3,4,5,6,7,8,9,10,11,12,13,14,15,16]
space['min_child_weight'] = [1, 5, 10, 20]
space['learning_rate'] = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.5]
space['subsample'] = [0.5, 0.7, 0.9, 1]
space['gamma'] = [0, 10, 50, 100]

# Define Search
search = GridSearchCV(model, space, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv_inner, verbose=0)
# Execute Search
search.fit(X_train, y_train)
print(search.best_params_)

# CV 
cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)

scores = cross_val_score(search, data, lable, scoring='neg_mean_absolute_error', cv=cv_outer, n_jobs=-1)

print((scores.mean(), scores.std()))


In [ ]:
## SVM grid search with repeated stratified kfold validation

data = PPdata[['RACE', 'CCI', 'SEX', 'AGE', 'TYPE_OF_ADMISSION', 'MEDICARE_STATUS_CODE']]

lable = PPdata['ICU']

model = SVR()
# define evaluation
cv_inner= KFold(n_splits=5, shuffle=True, random_state=42)
# define search space

space = dict()
space['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']
space['degree'] = [1,2,3,5]
space['C'] = [0.001, 0.01, 0.1, 1, 10, 100]
space['gamma'] = ['scale', 'auto']

# Define Search
search = GridSearchCV(model, space, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv_inner, verbose=0)
# Execute Search
search.fit(X_train, y_train)
print(search.best_params_) 

# CV
cv_outer = KFold(n_splits=10, shuffle=True, random_state=42)

scores = cross_val_score(search, data, lable, scoring='neg_mean_absolute_error', cv=cv_outer, n_jobs=-1)

print((scores.mean(), scores.std()))
